In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import datetime as dt


## What data is contained in a cookie?
1.  The name of the cookie.
2. The value of the cookie.
3. The expiration date of the cookie - this determines how long the cookie will remain active in your browser.
4. The path the cookie is valid for - this sets the URL path the cookie us valid in. Web pages outside of that path cannot use the cookie.
5. The domain the cookie is valid for. This makes the cookie accessible to pages on any of the servers when a site uses multiple servers in a domain.
6. The need for a secure connection - this indicates that the cookie can only be used under a secure server condition, such as a site using SSL.

In [2]:
header_columns=['domain', '???', 'path', '??', 'timestamp', 'name', 'value']

cookie_df = pd.read_csv('cookies.tsv', delimiter='\t', names=header_columns)

In [3]:
print cookie_df.nunique()
print len(cookie_df)

domain       1514
???             2
path          115
??              2
timestamp    3768
name         2399
value        3941
dtype: int64
4955


In [4]:
domains = cookie_df.domain.unique()

In [5]:
# write up a list of rules about the cookie paths 
"""
1. a '.' at the beginning of the strong
2. 'www.' at the beginning of the string
3. subdomains (i.e. .docs.google.com)
4. '.api.' at the beginning

Having looked through around 400 of them, it seems like the 
main problem is the subdomains. 
"""





"\n1. a '.' at the beginning of the strong\n2. 'www.' at the beginning of the string\n3. subdomains (i.e. .docs.google.com)\n4. '.api.' at the beginning\n\nHaving looked through around 400 of them, it seems like the \nmain problem is the subdomains. \n"

In [10]:
with open('suffix_list.txt') as fp:
    input_data = fp.read()

In [14]:
suffixes = set(input_data.split('\n'))

In [16]:
suffixes

{'',
 'chikuho.fukuoka.jp',
 '*.compute.amazonaws.com',
 'thruhere.net',
 '\xe5\xb9\xbf\xe4\xb8\x9c',
 'pvt.ge',
 'matsuzaki.shizuoka.jp',
 'name.eg',
 'tsuruga.fukui.jp',
 'in-addr.arpa',
 'hisayama.fukuoka.jp',
 'name.et',
 'botanicalgarden.museum',
 '\xe0\xa6\xad\xe0\xa6\xbe\xe0\xa7\xb0\xe0\xa6\xa4',
 'union.aero',
 'from-la.net',
 'mangyshlak.su',
 'dell',
 'biz.ki',
 'owani.aomori.jp',
 'kh.ua',
 'wallonie.museum',
 'medical.museum',
 'imb.br',
 '*.platformsh.site',
 'vald-aosta.it',
 'rochester.museum',
 'hisamitsu',
 'inf.ua',
 'seto.aichi.jp',
 'internet-dns.de',
 'pro.az',
 'nose.osaka.jp',
 'ayagawa.kagawa.jp',
 'esp.br',
 'okawa.fukuoka.jp',
 'dgca.aero',
 'nesoddtangen.no',
 'teaches-yoga.com',
 'cooking',
 'maibara.shiga.jp',
 'aigo',
 'kashihara.nara.jp',
 'dray-dns.de',
 'pd.it',
 'here',
 'lgbt',
 'kmpsp.gov.pl',
 'omuta.fukuoka.jp',
 'naumburg.museum',
 'association.aero',
 'sula.no',
 'nom.fr',
 'akkeshi.hokkaido.jp',
 'azumino.nagano.jp',
 'shiki.saitama.jp',
 'goldp

In [18]:
root_domain = []

In [29]:
test = '.docs.google.com'
test_split = test.split('.')
print test_split
test_split.reverse()
print test_split

for x in test_split:
    if x not in suffixes:
        root_domain.append(x)
        
        
        
print root_domain

['', 'docs', 'google', 'com']
['com', 'google', 'docs', '']
['splitwise']


In [24]:
def remove_subdomains(domain, suffixes):
    split = domain.split('.')
    split.reverse()
    for string in split:
        if string not in suffixes:
            return string
    

In [25]:
cookie_df['root_domain'] = [remove_subdomains(x, suffixes) for x in cookie_df.domain.values]

In [ ]:
# turns out that this is not a legit purpose

In [26]:
cookie_df.to_csv('cookie_df.csv')

In [25]:
dt.datetime.fromtimestamp(cookie_df.timestamp.values[0])

datetime.datetime(2018, 8, 21, 16, 19, 48)

In [33]:
human_readable_dates = []

In [34]:
for x in cookie_df.timestamp.values:
    try: 
        human_readable_dates.append(dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    except ValueError:
        human_readable_dates.append(dt.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
        print x
        print dt.datetime.fromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
        print "had to divide by 1000"

# cookie_df['ts'] = [dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S') for x in cookie_df.timestamp.values]

1512347537475
2017-12-04 00:32:17
had to divide by 1000
253402300800
1978-01-11 21:31:40
had to divide by 1000


In [37]:
len(human_readable_dates)
cookie_df['ts'] = human_readable_dates

In [7]:
# cookie_df.to_sql()

TypeError: to_sql() takes at least 3 arguments (1 given)

In [65]:
rcur = pymysql.connect(host='localhost',user='root', database='glassdb')
# engine = create_engine('mysql://root:@localhost:3306/glassdb')

In [64]:
# cookie_df.to_sql('cookies', engine, if_exists='replace')
rcur.close()

In [19]:
cols = cookie_df.columns.values
tuple(cols)

('domain', '???', 'path', '??', 'timestamp', 'name', 'value')

In [38]:
create_table_query = """CREATE TABLE seed_cookies
    (domain varchar(255),
    mystery_1 boolean,
    valid_path varchar(255),
    mystery_2 boolean,
    unix_time long,
    name varchar(255), 
    value varchar(10000),
    expiry_date datetime
    );"""

In [66]:
tuples = str([tuple(x) for x in cookie_df.fillna('').values])[1:-1]


In [67]:
insert_data_query = """INSERT INTO seed_cookies values {tuples}""".format(tuples=tuples)

# print insert_data_query

In [68]:
rcur.execute(insert_data_query)

AttributeError: 'Connection' object has no attribute 'execute'

In [71]:
# rcur.cursor().execute(insert_data_query)
rcur.commit()
rcur.close()

array(['.metacritic.com', '.techcrunch.com', 'www.telegraph.co.uk', ...,
       '.iu.edu', 'www.whatarecookies.com', '.whatarecookies.com'], dtype=object)